# Store Sales Forecasting with RNNs 📈📉

## Introduction ✏️

Time series forecasting is one of the most important tasks in the world of business. It is a very complex task, and it is not always possible to predict the future. But we can build ML models to do so. One of the best ways to do so is to use recurrent neural networks (RNNs), which can handle time series data pretty well because they keep a memory state of the previous time steps.

To apply this concept, we will use the [Store Sales - Time Series Forecasting](https://www.kaggle.com/c/store-sales-time-series-forecasting/data) to predict the sales of a store in the next two weeks. We will read, manipulate and visualize the data, and then build a model to predict the sales. Let's get started!

<h1>Table of Contents 📕<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-2">Dependencies</a></span></li><li><span><a href="#Reading-Data" data-toc-modified-id="Reading-Data-3">Reading Data</a></span><ul class="toc-item"><li><span><a href="#Train-Data" data-toc-modified-id="Train-Data-3.1">Train Data</a></span></li><li><span><a href="#Test-Data" data-toc-modified-id="Test-Data-3.2">Test Data</a></span></li><li><span><a href="#Sample-Submission" data-toc-modified-id="Sample-Submission-3.3">Sample Submission</a></span></li><li><span><a href="#Stores-Data" data-toc-modified-id="Stores-Data-3.4">Stores Data</a></span></li><li><span><a href="#Oil-Data" data-toc-modified-id="Oil-Data-3.5">Oil Data</a></span></li><li><span><a href="#Holiday-Events-Data" data-toc-modified-id="Holiday-Events-Data-3.6">Holiday Events Data</a></span></li><li><span><a href="#Additional-Notes" data-toc-modified-id="Additional-Notes-3.7">Additional Notes</a></span></li></ul></li><li><span><a href="#Understanding-the-Data" data-toc-modified-id="Understanding-the-Data-4">Understanding the Data</a></span><ul class="toc-item"><li><span><a href="#Merging-our-tables" data-toc-modified-id="Merging-our-tables-4.1">Merging our tables</a></span></li></ul></li></ul></div>

## Dependencies 👪

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os

## Reading Data 📖

In [2]:
print("Path of our data files:")
for path in os.listdir("./data"):
    print(os.path.join("./data", path))

Path of our data files:
./data\holidays_events.csv
./data\oil.csv
./data\sample_submission.csv
./data\stores.csv
./data\test.csv
./data\train.csv
./data\transactions.csv


### Train Data

- The training data, comprising time series of features `store_nbr`, `family`, and `onpromotion` as well as the target `sales`.
- `store_nbr` identifies the store at which the products are sold.
- `family` identifies the type of product sold.
- `sales` gives the total sales for a product `family` at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- `onpromotion` gives the total number of items in a product family that were being promoted at a store at a given date.

In [3]:
train_data = pd.read_csv("./data/train.csv", index_col="id")
train_data.head()

C:\Users\ubfil\anaconda3\envs\ai-env\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


### Test Data

- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.

In [4]:
test_data = pd.read_csv("./data/test.csv", index_col="id")
test_data.head()

,date,store_nbr,family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2
3000891,2017-08-16,1,BEVERAGES,20
3000892,2017-08-16,1,BOOKS,0


### Sample Submission

- A sample submission file in the correct format.

In [5]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


### Stores Data

- Store metadata, including `city`, `state`, `type`, and `cluster`.
- `cluster` is a grouping of similar stores.

In [6]:
stores_data = pd.read_csv("./data/stores.csv")
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Oil Data

- Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [7]:
oil_data = pd.read_csv("./data/oil.csv")
oil_data.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### Transactions Data

- Number of complete daily web transactions for each store.

In [8]:
transactions_data = pd.read_csv("./data/transactions.csv")
transactions_data.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Holiday Events Data

- Holidays and Events, with metadata
- **NOTE**: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [9]:
holidays_data = pd.read_csv("./data/holidays_events.csv")
holidays_data.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### Additional Notes

- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

All these data descriptions are not mine, they were provided by the Kaggle team. I put them here so we can have a better understanding of the data. You can check it out in the [Kaggle competition page](https://www.kaggle.com/c/store-sales-time-series-forecasting/data).

## Understanding the Data 📚
In this section, we will look at the data and see what we can learn from it. We will merge different tables, create new features, and explore the data with different visualizations.

Firstly, let's merge our tables.

### Merging our tables

We will merge our training/test data with our store metadata, using the `store_nbr` column as the key.

#### Store metadata

In [10]:
train_data = pd.merge(train_data, stores_data, on="store_nbr")
test_data = pd.merge(test_data, stores_data, on="store_nbr")
train_data.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


Now, let's do the same with our oil data. But firstly, let's see if our data has too many NaN values.

#### Oil data

In [11]:
print("Shape of oil_data:" ,oil_data.shape)

Shape of oil_data: (1218, 2)


In [12]:
print("Number of null values in oil_data:")
oil_data.isnull().sum()

Number of null values in oil_data:


date           0
dcoilwtico    43
dtype: int64

We have only 43 NaN values in the oil data. But if we merge our table with the oil data, we will have way more NaN values. Let's analyze the best way to impute the NaN values and then merge our tables.

In [13]:
fig = go.Figure(go.Scatter(x=oil_data.date, y=oil_data.dcoilwtico, mode="lines", marker_color="brown"))
fig.update_xaxes(
    showgrid=True,
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ]),
        font_color="black"
    )
) # adding range slider
fig.update_layout(yaxis_title="Price",
                  yaxis_showgrid=False,
                  template="plotly_white", 
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  margin=dict(b=20,r=50,l=70,t=115),
                  title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>from 2013 to 2016<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                  font=dict(color='#666'),
                 )
fig.show()

We can see that besides the oil price is unstable, the oil price is decreasing, and besides some NaN values, we can see a continuous curve along time. So we can impute the NaN values with the last known value, to maintain our continuity.

In [14]:
oil_data.fillna(method="bfill", inplace=True)

In [15]:
fig = go.Figure(go.Scatter(x=oil_data.date, y=oil_data.dcoilwtico, mode="lines"))
fig.update_xaxes(
    title_text="Date",
    showgrid=False,
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ]),
        font_color="black"
    )
)
fig.update_layout(yaxis_title="Price",
                  yaxis_showgrid=False,
                  template="plotly_white", 
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  margin=dict(b=20,r=50,l=70,t=115),
                  title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>with replaced <span style='color:#6D83AA'>NaN </span>values<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                  font=dict(color='#666'),
                 )
fig.show()

In [16]:
train_data = pd.merge(train_data, oil_data, on="date")
test_data = pd.merge(test_data, oil_data, on="date")

train_data.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


#### Transactions

In [17]:
print("nº of null values in transactions_data:")
transactions_data.isnull().sum()

nº of null values in transactions_data:


date            0
store_nbr       0
transactions    0
dtype: int64

In [18]:
train_data = pd.merge(train_data, transactions_data, on=["date", "store_nbr"])
test_data = pd.merge(test_data, transactions_data, on=["date", "store_nbr"])

train_data.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,2013-01-01,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,93.14,770
1,2013-01-01,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,93.14,770
2,2013-01-01,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,93.14,770
3,2013-01-01,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,93.14,770
4,2013-01-01,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,93.14,770


### Holiday Events

In [19]:
print("nº of null values in holidays_data:")
holidays_data.isnull().sum()

nº of null values in holidays_data:


date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [20]:
train_data = pd.merge(train_data, holidays_data, on=["date"])
test_data = pd.merge(test_data, holidays_data, on=["date"])

train_data.head()

,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,transactions,type_y,locale,locale_name,description,transferred
0,2013-01-01,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,93.14,770,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-01,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,93.14,770,Holiday,National,Ecuador,Primer dia del ano,False
2,2013-01-01,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,93.14,770,Holiday,National,Ecuador,Primer dia del ano,False
3,2013-01-01,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,93.14,770,Holiday,National,Ecuador,Primer dia del ano,False
4,2013-01-01,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,93.14,770,Holiday,National,Ecuador,Primer dia del ano,False


Now that we have just one table, let's analyze what columns are useful and gain insights about our data.

## Analyzing our Data 📊

### Does Oil Price really affects sales?
Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices. But does this reflects in the sales of our products? 

In [21]:
# converting date to datetime
train_data.date = pd.to_datetime(train_data.date)
test_data.date = pd.to_datetime(test_data.date)

In [22]:
def month_and_year(date):
    """_summary_
    This function returns the month and year of a given date
    
    Args:
        date (dt.datetime): date
    """
    
    return date.strftime("%b-%Y")

train_data["month_and_year"] = train_data.date.apply(month_and_year) # will be used to find mean sales per month
sales_oil_by_month = train_data[["month_and_year", "sales", "dcoilwtico"]].groupby(train_data.month_and_year).mean()

In [32]:
sales_oil_by_month = train_data[["date", "sales", "dcoilwtico"]].copy()
sales_oil_by_month = sales_oil_by_month.groupby(pd.Grouper(key="date", freq="M")).mean()
sales_oil_by_month.fillna(method="ffill", inplace=True)

In [76]:
fig = make_subplots(rows=2, cols=1)
# adding traces
fig.add_trace(go.Scatter(x=sales_oil_by_month.index, y=sales_oil_by_month.sales, mode="lines", name="Sales"), row=1, col=1)
fig.add_trace(go.Scatter(x=sales_oil_by_month.index, y=sales_oil_by_month.dcoilwtico, mode="lines", name="Oil Price"), row=2, col=1)
# updating axis titles
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Sales", row=1, col=1)
fig.update_yaxes(title_text="Oil Price", row=2, col=1)
# add vertical line in the plot
fig.update_layout(
        shapes=[
        dict(type="line", x0=pd.to_datetime("2014-12-31"), xref="x", y0=0, yref="y", x1=pd.to_datetime("2014-12-31"),
        y1=sales_oil_by_month.sales.max(),
        line=dict(color="black", width=2, dash="dot")),
        dict(type="line", x0=pd.to_datetime("2014-12-31"), xref="x2", y0=0, yref="y2", x1=pd.to_datetime("2014-12-31"),
        y1=sales_oil_by_month.dcoilwtico.max(),
        line=dict(color="black", width=2, dash="dot")),
        ],
        
        annotations=[
                dict(x=pd.to_datetime("2015-10-31"), y=sales_oil_by_month.sales.max()+100, xref="x", yref="y",
        text="When Oil Price starts to decrease, sales start to increase", showarrow=False, font=dict(color="black", size=12))
        ]
)
# add annotation
fig.update_layout(
        )
# updating figure layout
fig.update_layout(template="plotly_white",
                    plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                    margin=dict(b=20,r=60,l=70,t=115),
                    title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Sales and Oil Price</span><br>from 2013 to 2016<b><br><br>",
                            'y':0.85, 'x':0.5,
                            'xanchor': 'center', 'yanchor': 'top'},
                    font=dict(color='#666'),
                    )
fig.show()

In [24]:
fig = go.Figure()
fig.add_trace(go.Bar(x=sales_oil_by_month.index, y=sales_oil_by_month.sales,name="Sales", marker_color='#6D83AA'))
fig.add_trace(go.Scatter(x=sales_oil_by_month.index, y=sales_oil_by_month.dcoilwtico, mode="lines", name="Oil Price",
                         marker_color='#FFA500'))

fig.update_layout(yaxis_title="Price",
                  yaxis_showgrid=False,
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  margin=dict(b=20,r=50,l=70,t=115),
                  #title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>with replaced <span style='color:#6D83AA'>NaN </span>values<b>",
                  #       'y':0.85, 'x':0.5, 
                  #       'xanchor': 'center', 'yanchor': 'top'},
                  font=dict(color='#666'),
                 )
fig.show()

# Notes

- Save csv file of preprocessed data.

## Questions
1. Does Oil Price really affects Sales?
2. What are the stores that sell more? Does location matter?
3. Amount and total of sales per family?
4. Try to gather population by city and state.
5. How many stores are in each city?
6. What about holidays? Which are the most important? How can we change the date of holidays that were transferred?
7. Description of holidays, locale, and locale_name, can we find out more about the holidays?
8. Does promotion relates to oil price?